# Packages

In [3]:
import pandas as pd
from pathlib import Path
import yaml
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

# Statics

In [5]:
MAIN_PATH = Path("/".join(os.getcwd().split("/")[:-1]))
REPO_PATH = MAIN_PATH / "customerone-w" 
print(REPO_PATH)

/Users/franco_zentilli/Desktop/python/customerone-w


# Utils

In [70]:
def get_globals(repo_path, realm, base_path_name="base_path"):
    globals_paths = {}
    realm_path = repo_path / "realm" / realm
    with open(realm_path / "conf/globals/dev.yaml") as file:
        yaml_file = yaml.load(file, Loader=yaml.FullLoader)
    base_path = yaml_file[base_path_name]
    var_sffx = "${.%s}" % base_path_name
    for k in yaml_file:
        globals_paths[k] = yaml_file[k].replace(var_sffx, base_path)
    
    return globals_paths

# test
get_globals_paths(repo_path=REPO_PATH, realm="rb")

{'base_path': 'data',
 'input_dir_path': 'data/input',
 'fs_dir_path': 'data/feature_store',
 'uc_dir_path': 'data/use_case'}

In [117]:
def get_data_paths_from_catalog(globals_paths, catalog_path):
    """
        catalog_path: path.to.catalog.yaml
    """
    data_paths = {}
    catalog_dir_path = catalog_path.parent
    with open(catalog_path) as file:
        yaml_file = yaml.load(file, Loader=yaml.FullLoader)
        
    for k0, its0 in yaml_file.items():
        for k1, its1 in its0.items():
            if k1 == "filepath":
                its1_splits = its1.split("/")
                sffx = its1_splits[0]
                for g in globals_paths:
                    if g in sffx:
                        data_paths[k0.split("@")[0]] = "{}/{}".format(globals_paths[g],"/".join(its1_splits[1:]))
    
    return data_paths

# test
globals_paths = get_globals_paths(repo_path=REPO_PATH, realm="rb")
catalog_path = REPO_PATH / "realm/rb/conf/base/fs/catalog.yaml"
get_data_paths_from_catalog(globals_paths, catalog_path)

{'credit_card': 'data/feature_store/credit_card',
 'current_account': 'data/feature_store/current_account',
 'debit_card': 'data/feature_store/debit_card',
 'saving_account': 'data/feature_store/saving_account',
 'customer': 'data/feature_store/customer',
 'churn': 'data/feature_store/churn',
 'loan': 'data/feature_store/loan'}

In [137]:
def extensive_describe(table):
    print("Display")
    display(table.head())
    for t in ("object", None):
        print("Type: {}".format("numeric" if t is None else t))
        desc = table.describe(include=t)
        desc.loc['dtype'] = table.dtypes
        desc.loc['size'] = len(table)
        desc.loc['nan %'] = table.isnull().mean()
        display(desc)
    return table
extensive_describe(table)

Display


,_id,_observ_end_dt,customer_id,_observ,_observ_start_dt,cc_customer_exists_flag,cc_credit_card_cnt,cc_sum_credit_card_limit_val,cc_sum_rotation_credit_used_val,cc_nb_stolen_cards_val,cc_nb_blocked_cards_val,cc_product_name_typ,cc_utilization_credit_limit_val,prev_cc_product_name_typ,cc_product_change_typ,cc_black_cnt,cc_classic_cnt,cc_gold_cnt,cc_platinum_cnt,has_black_flag,has_classic_flag,has_gold_flag,has_platinum_flag,cc_black_black_flag,cc_black_classic_flag,cc_black_gold_flag,cc_black_platinum_flag,cc_classic_black_flag,cc_classic_classic_flag,cc_classic_gold_flag,cc_classic_platinum_flag,cc_gold_black_flag,cc_gold_classic_flag,cc_gold_gold_flag,cc_gold_platinum_flag,cc_platinum_black_flag,cc_platinum_classic_flag,cc_platinum_gold_flag,cc_platinum_platinum_flag,cc_avg_transaction_amount_val,cc_sum_transaction_amount_val,cc_transaction_cnt,bpg_services_mcc_cc_avg_transaction_amount_val,bpg_services_mcc_cc_sum_transaction_amount_val,bpg_services_mcc_cc_transaction_cnt,commercial_services_mcc_cc_avg_transaction_amount_val,commercial_services_mcc_cc_sum_transaction_amount_val,commercial_services_mcc_cc_transaction_cnt,stores_mcc_cc_avg_transaction_amount_val,stores_mcc_cc_sum_transaction_amount_val,stores_mcc_cc_transaction_cnt,travel_mcc_cc_avg_transaction_amount_val,travel_mcc_cc_sum_transaction_amount_val,travel_mcc_cc_transaction_cnt,cc_first_reference_dt,has_first_cc_flag,cc_first_reference_dt_flag,cc_second_reference_dt,has_second_cc_flag,cc_second_reference_dt_flag,cc_hard_churn_flag,cc_hard_churn_reference_dt,has_hard_churn_cc_flag,cc_hard_churn_reference_dt_flag,cc_sum_transaction_fee_val,cc_ltv_reference_dt,cc_ltv_reference_dt_flag
0,cuid_000001,2015-01-31,cuid_000001,0,2015-01-01,1,1,1000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
1,cuid_000001,2015-02-28,cuid_000001,1,2015-02-01,1,1,5000,0.000000,0,0,platinum,0.000000,classic,classic_platinum,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
2,cuid_000001,2015-03-31,cuid_000001,2,2015-03-01,1,1,2000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
3,cuid_000001,2015-04-30,cuid_000001,3,2015-04-01,1,1,5000,0.000000,0,0,classic,0.000000,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
4,cuid_000001,2015-05-31,cuid_000001,4,2015-05-01,1,1,5000,4301.925916,0,0,classic,86.038518,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0


Type: object


,_id,_observ_end_dt,customer_id,_observ_start_dt,cc_product_name_typ,prev_cc_product_name_typ,cc_product_change_typ,cc_first_reference_dt,cc_second_reference_dt,cc_hard_churn_reference_dt,cc_ltv_reference_dt
count,8800,8800,8800,8800,4015,4015,4015,7206,7160,3478,5534
unique,100,88,100,88,4,4,16,16,6,9,16
top,cuid_000001,2015-01-31,cuid_000001,2015-01-01,platinum,platinum,platinum_platinum,2022-04-30,2022-04-30,2022-04-30,2022-04-30
freq,88,100,88,100,1698,1743,858,7128,7154,3470,5456
dtype,object,object,object,object,object,object,object,object,object,object,object
size,8800,8800,8800,8800,8800,8800,8800,8800,8800,8800,8800
nan %,0.0,0.0,0.0,0.0,0.54375,0.54375,0.54375,0.181136,0.186364,0.604773,0.371136


Type: numeric


,_observ,cc_customer_exists_flag,cc_credit_card_cnt,cc_sum_credit_card_limit_val,cc_sum_rotation_credit_used_val,cc_nb_stolen_cards_val,cc_nb_blocked_cards_val,cc_utilization_credit_limit_val,cc_black_cnt,cc_classic_cnt,cc_gold_cnt,cc_platinum_cnt,has_black_flag,has_classic_flag,has_gold_flag,has_platinum_flag,cc_black_black_flag,cc_black_classic_flag,cc_black_gold_flag,cc_black_platinum_flag,cc_classic_black_flag,cc_classic_classic_flag,cc_classic_gold_flag,cc_classic_platinum_flag,cc_gold_black_flag,cc_gold_classic_flag,cc_gold_gold_flag,cc_gold_platinum_flag,cc_platinum_black_flag,cc_platinum_classic_flag,cc_platinum_gold_flag,cc_platinum_platinum_flag,cc_avg_transaction_amount_val,cc_sum_transaction_amount_val,cc_transaction_cnt,bpg_services_mcc_cc_avg_transaction_amount_val,bpg_services_mcc_cc_sum_transaction_amount_val,bpg_services_mcc_cc_transaction_cnt,commercial_services_mcc_cc_avg_transaction_amount_val,commercial_services_mcc_cc_sum_transaction_amount_val,commercial_services_mcc_cc_transaction_cnt,stores_mcc_cc_avg_transaction_amount_val,stores_mcc_cc_sum_transaction_amount_val,stores_mcc_cc_transaction_cnt,travel_mcc_cc_avg_transaction_amount_val,travel_mcc_cc_sum_transaction_amount_val,travel_mcc_cc_transaction_cnt,has_first_cc_flag,cc_first_reference_dt_flag,has_second_cc_flag,cc_second_reference_dt_flag,cc_hard_churn_flag,has_hard_churn_cc_flag,cc_hard_churn_reference_dt_flag,cc_sum_transaction_fee_val,cc_ltv_reference_dt_flag
count,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0
mean,43.5,0.45625,0.950568,2558.068182,809.70532,0.016705,0.150909,19.060848,0.234318,0.242614,0.232045,0.241591,0.185341,0.192273,0.186477,0.192955,0.011364,0.012841,0.014205,0.017273,0.013636,0.018295,0.022386,0.031477,0.015227,0.02375,0.031023,0.046705,0.016932,0.032386,0.05125,0.0975,8.573295,142.642126,0.147273,6.001553,32.295477,0.032727,5.860368,56.771575,0.060341,5.335446,22.065826,0.021591,5.296644,31.509248,0.032614,0.008864,0.011364,0.000682,0.009773,0.000909,0.000909,0.000909,0.141419,0.009205
std,25.403215,0.498111,1.354174,4015.742507,1855.222661,0.129931,0.416938,55.097911,0.54597,0.552199,0.537085,0.548622,0.388596,0.394109,0.389513,0.39464,0.105999,0.112594,0.11834,0.130293,0.115983,0.134025,0.147945,0.174614,0.122463,0.152278,0.173389,0.211017,0.129023,0.177034,0.22052,0.296654,105.044219,2045.883072,2.076203,87.079386,494.277046,0.480862,82.633998,850.170759,0.871892,81.081891,368.987119,0.33269,79.317192,515.40398,0.487559,0.093734,0.105999,0.026104,0.098378,0.030139,0.030139,0.030139,1.997087,0.095503
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-483.559497,-483.559497,0.0,-449.65219,-449.65219,0.0,-483.559497,-483.559497,0.0,0.0,0.0,0.0,-203.175383,-203.175383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,21.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,43.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,65.25,1.0,2.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,87.0,1.0,8.0,27000.0,19482.264368,2.0,4.0,499.910109,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0

,_id,_observ_end_dt,customer_id,_observ,_observ_start_dt,cc_customer_exists_flag,cc_credit_card_cnt,cc_sum_credit_card_limit_val,cc_sum_rotation_credit_used_val,cc_nb_stolen_cards_val,cc_nb_blocked_cards_val,cc_product_name_typ,cc_utilization_credit_limit_val,prev_cc_product_name_typ,cc_product_change_typ,cc_black_cnt,cc_classic_cnt,cc_gold_cnt,cc_platinum_cnt,has_black_flag,has_classic_flag,has_gold_flag,has_platinum_flag,cc_black_black_flag,cc_black_classic_flag,cc_black_gold_flag,cc_black_platinum_flag,cc_classic_black_flag,cc_classic_classic_flag,cc_classic_gold_flag,cc_classic_platinum_flag,cc_gold_black_flag,cc_gold_classic_flag,cc_gold_gold_flag,cc_gold_platinum_flag,cc_platinum_black_flag,cc_platinum_classic_flag,cc_platinum_gold_flag,cc_platinum_platinum_flag,cc_avg_transaction_amount_val,cc_sum_transaction_amount_val,cc_transaction_cnt,bpg_services_mcc_cc_avg_transaction_amount_val,bpg_services_mcc_cc_sum_transaction_amount_val,bpg_services_mcc_cc_transaction_cnt,commercial_services_mcc_cc_avg_transaction_amount_val,commercial_services_mcc_cc_sum_transaction_amount_val,commercial_services_mcc_cc_transaction_cnt,stores_mcc_cc_avg_transaction_amount_val,stores_mcc_cc_sum_transaction_amount_val,stores_mcc_cc_transaction_cnt,travel_mcc_cc_avg_transaction_amount_val,travel_mcc_cc_sum_transaction_amount_val,travel_mcc_cc_transaction_cnt,cc_first_reference_dt,has_first_cc_flag,cc_first_reference_dt_flag,cc_second_reference_dt,has_second_cc_flag,cc_second_reference_dt_flag,cc_hard_churn_flag,cc_hard_churn_reference_dt,has_hard_churn_cc_flag,cc_hard_churn_reference_dt_flag,cc_sum_transaction_fee_val,cc_ltv_reference_dt,cc_ltv_reference_dt_flag
0,cuid_000001,2015-01-31,cuid_000001,0,2015-01-01,1,1,1000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
1,cuid_000001,2015-02-28,cuid_000001,1,2015-02-01,1,1,5000,0.000000,0,0,platinum,0.000000,classic,classic_platinum,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
2,cuid_000001,2015-03-31,cuid_000001,2,2015-03-01,1,1,2000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
3,cuid_000001,2015-04-30,cuid_000001,3,2015-04-01,1,1,5000,0.000000,0,0,classic,0.000000,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
4,cuid_000001,2015-05-31,cuid_000001,4,2015-05-01,1,1,5000,4301.925916,0,0,classic,86.038518,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,cuid_000100,2021-12-31,cuid_000100,83,2021-12-01,1,1,2000,0.000000,0,0,classic,0.000000,black,black_classic,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,None,0,0,None,0,0,0,2022-04-30,0,0,0.0,None,0
8796,cuid_000100,2022-01-31,cuid_000100,84,2022-01-01,1,1,2000,0.000000,0,0,black,0.000000,classic,classic_black,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,None,0,0,None,0,0,0,2022-04-30,0,0,0.0,None,0
8797,cuid_000100,2022-02-28,cuid_000100,85,2022-02-01,1,1,2000,0.000000,0,0,gold,0.000000,black,black_gold,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [138]:
def get_table_from_catalog(catalog_path, repo_path=None, table_name=None, realm=None, summary=True):
    realms = ["rb", "cr"]
    table = pd.DataFrame()
    
    if realm is None:
        for r in realms:
            if r in catalog_path.parts:
                realm=r
    realm_path = repo_path / "realm/{}".format(realm)
    globals_paths = get_globals_paths(repo_path=repo_path, realm=realm)
    data_paths = get_data_paths_from_catalog(globals_paths=globals_paths, catalog_path=catalog_path)
    data_path = realm_path / data_paths[table_name]
    table = pd.read_parquet(data_path)
    
    if summary:
        extensive_describe(table)
        
    
    return table

# test
catalog_path = REPO_PATH / "realm/rb/conf/base/fs/catalog.yaml"
table_name = "credit_card"
table = get_table_from_catalog(catalog_path=catalog_path, repo_path=REPO_PATH, table_name=table_name)
table

Display


,_id,_observ_end_dt,customer_id,_observ,_observ_start_dt,cc_customer_exists_flag,cc_credit_card_cnt,cc_sum_credit_card_limit_val,cc_sum_rotation_credit_used_val,cc_nb_stolen_cards_val,cc_nb_blocked_cards_val,cc_product_name_typ,cc_utilization_credit_limit_val,prev_cc_product_name_typ,cc_product_change_typ,cc_black_cnt,cc_classic_cnt,cc_gold_cnt,cc_platinum_cnt,has_black_flag,has_classic_flag,has_gold_flag,has_platinum_flag,cc_black_black_flag,cc_black_classic_flag,cc_black_gold_flag,cc_black_platinum_flag,cc_classic_black_flag,cc_classic_classic_flag,cc_classic_gold_flag,cc_classic_platinum_flag,cc_gold_black_flag,cc_gold_classic_flag,cc_gold_gold_flag,cc_gold_platinum_flag,cc_platinum_black_flag,cc_platinum_classic_flag,cc_platinum_gold_flag,cc_platinum_platinum_flag,cc_avg_transaction_amount_val,cc_sum_transaction_amount_val,cc_transaction_cnt,bpg_services_mcc_cc_avg_transaction_amount_val,bpg_services_mcc_cc_sum_transaction_amount_val,bpg_services_mcc_cc_transaction_cnt,commercial_services_mcc_cc_avg_transaction_amount_val,commercial_services_mcc_cc_sum_transaction_amount_val,commercial_services_mcc_cc_transaction_cnt,stores_mcc_cc_avg_transaction_amount_val,stores_mcc_cc_sum_transaction_amount_val,stores_mcc_cc_transaction_cnt,travel_mcc_cc_avg_transaction_amount_val,travel_mcc_cc_sum_transaction_amount_val,travel_mcc_cc_transaction_cnt,cc_first_reference_dt,has_first_cc_flag,cc_first_reference_dt_flag,cc_second_reference_dt,has_second_cc_flag,cc_second_reference_dt_flag,cc_hard_churn_flag,cc_hard_churn_reference_dt,has_hard_churn_cc_flag,cc_hard_churn_reference_dt_flag,cc_sum_transaction_fee_val,cc_ltv_reference_dt,cc_ltv_reference_dt_flag
0,cuid_000001,2015-01-31,cuid_000001,0,2015-01-01,1,1,1000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
1,cuid_000001,2015-02-28,cuid_000001,1,2015-02-01,1,1,5000,0.000000,0,0,platinum,0.000000,classic,classic_platinum,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
2,cuid_000001,2015-03-31,cuid_000001,2,2015-03-01,1,1,2000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
3,cuid_000001,2015-04-30,cuid_000001,3,2015-04-01,1,1,5000,0.000000,0,0,classic,0.000000,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
4,cuid_000001,2015-05-31,cuid_000001,4,2015-05-01,1,1,5000,4301.925916,0,0,classic,86.038518,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0


Type: object


,_id,_observ_end_dt,customer_id,_observ_start_dt,cc_product_name_typ,prev_cc_product_name_typ,cc_product_change_typ,cc_first_reference_dt,cc_second_reference_dt,cc_hard_churn_reference_dt,cc_ltv_reference_dt
count,8800,8800,8800,8800,4015,4015,4015,7206,7160,3478,5534
unique,100,88,100,88,4,4,16,16,6,9,16
top,cuid_000001,2015-01-31,cuid_000001,2015-01-01,platinum,platinum,platinum_platinum,2022-04-30,2022-04-30,2022-04-30,2022-04-30
freq,88,100,88,100,1698,1743,858,7128,7154,3470,5456
dtype,object,object,object,object,object,object,object,object,object,object,object
size,8800,8800,8800,8800,8800,8800,8800,8800,8800,8800,8800
nan %,0.0,0.0,0.0,0.0,0.54375,0.54375,0.54375,0.181136,0.186364,0.604773,0.371136


Type: numeric


,_observ,cc_customer_exists_flag,cc_credit_card_cnt,cc_sum_credit_card_limit_val,cc_sum_rotation_credit_used_val,cc_nb_stolen_cards_val,cc_nb_blocked_cards_val,cc_utilization_credit_limit_val,cc_black_cnt,cc_classic_cnt,cc_gold_cnt,cc_platinum_cnt,has_black_flag,has_classic_flag,has_gold_flag,has_platinum_flag,cc_black_black_flag,cc_black_classic_flag,cc_black_gold_flag,cc_black_platinum_flag,cc_classic_black_flag,cc_classic_classic_flag,cc_classic_gold_flag,cc_classic_platinum_flag,cc_gold_black_flag,cc_gold_classic_flag,cc_gold_gold_flag,cc_gold_platinum_flag,cc_platinum_black_flag,cc_platinum_classic_flag,cc_platinum_gold_flag,cc_platinum_platinum_flag,cc_avg_transaction_amount_val,cc_sum_transaction_amount_val,cc_transaction_cnt,bpg_services_mcc_cc_avg_transaction_amount_val,bpg_services_mcc_cc_sum_transaction_amount_val,bpg_services_mcc_cc_transaction_cnt,commercial_services_mcc_cc_avg_transaction_amount_val,commercial_services_mcc_cc_sum_transaction_amount_val,commercial_services_mcc_cc_transaction_cnt,stores_mcc_cc_avg_transaction_amount_val,stores_mcc_cc_sum_transaction_amount_val,stores_mcc_cc_transaction_cnt,travel_mcc_cc_avg_transaction_amount_val,travel_mcc_cc_sum_transaction_amount_val,travel_mcc_cc_transaction_cnt,has_first_cc_flag,cc_first_reference_dt_flag,has_second_cc_flag,cc_second_reference_dt_flag,cc_hard_churn_flag,has_hard_churn_cc_flag,cc_hard_churn_reference_dt_flag,cc_sum_transaction_fee_val,cc_ltv_reference_dt_flag
count,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0
mean,43.5,0.45625,0.950568,2558.068182,809.70532,0.016705,0.150909,19.060848,0.234318,0.242614,0.232045,0.241591,0.185341,0.192273,0.186477,0.192955,0.011364,0.012841,0.014205,0.017273,0.013636,0.018295,0.022386,0.031477,0.015227,0.02375,0.031023,0.046705,0.016932,0.032386,0.05125,0.0975,8.573295,142.642126,0.147273,6.001553,32.295477,0.032727,5.860368,56.771575,0.060341,5.335446,22.065826,0.021591,5.296644,31.509248,0.032614,0.008864,0.011364,0.000682,0.009773,0.000909,0.000909,0.000909,0.141419,0.009205
std,25.403215,0.498111,1.354174,4015.742507,1855.222661,0.129931,0.416938,55.097911,0.54597,0.552199,0.537085,0.548622,0.388596,0.394109,0.389513,0.39464,0.105999,0.112594,0.11834,0.130293,0.115983,0.134025,0.147945,0.174614,0.122463,0.152278,0.173389,0.211017,0.129023,0.177034,0.22052,0.296654,105.044219,2045.883072,2.076203,87.079386,494.277046,0.480862,82.633998,850.170759,0.871892,81.081891,368.987119,0.33269,79.317192,515.40398,0.487559,0.093734,0.105999,0.026104,0.098378,0.030139,0.030139,0.030139,1.997087,0.095503
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-483.559497,-483.559497,0.0,-449.65219,-449.65219,0.0,-483.559497,-483.559497,0.0,0.0,0.0,0.0,-203.175383,-203.175383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,21.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,43.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,65.25,1.0,2.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,87.0,1.0,8.0,27000.0,19482.264368,2.0,4.0,499.910109,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0

,_id,_observ_end_dt,customer_id,_observ,_observ_start_dt,cc_customer_exists_flag,cc_credit_card_cnt,cc_sum_credit_card_limit_val,cc_sum_rotation_credit_used_val,cc_nb_stolen_cards_val,cc_nb_blocked_cards_val,cc_product_name_typ,cc_utilization_credit_limit_val,prev_cc_product_name_typ,cc_product_change_typ,cc_black_cnt,cc_classic_cnt,cc_gold_cnt,cc_platinum_cnt,has_black_flag,has_classic_flag,has_gold_flag,has_platinum_flag,cc_black_black_flag,cc_black_classic_flag,cc_black_gold_flag,cc_black_platinum_flag,cc_classic_black_flag,cc_classic_classic_flag,cc_classic_gold_flag,cc_classic_platinum_flag,cc_gold_black_flag,cc_gold_classic_flag,cc_gold_gold_flag,cc_gold_platinum_flag,cc_platinum_black_flag,cc_platinum_classic_flag,cc_platinum_gold_flag,cc_platinum_platinum_flag,cc_avg_transaction_amount_val,cc_sum_transaction_amount_val,cc_transaction_cnt,bpg_services_mcc_cc_avg_transaction_amount_val,bpg_services_mcc_cc_sum_transaction_amount_val,bpg_services_mcc_cc_transaction_cnt,commercial_services_mcc_cc_avg_transaction_amount_val,commercial_services_mcc_cc_sum_transaction_amount_val,commercial_services_mcc_cc_transaction_cnt,stores_mcc_cc_avg_transaction_amount_val,stores_mcc_cc_sum_transaction_amount_val,stores_mcc_cc_transaction_cnt,travel_mcc_cc_avg_transaction_amount_val,travel_mcc_cc_sum_transaction_amount_val,travel_mcc_cc_transaction_cnt,cc_first_reference_dt,has_first_cc_flag,cc_first_reference_dt_flag,cc_second_reference_dt,has_second_cc_flag,cc_second_reference_dt_flag,cc_hard_churn_flag,cc_hard_churn_reference_dt,has_hard_churn_cc_flag,cc_hard_churn_reference_dt_flag,cc_sum_transaction_fee_val,cc_ltv_reference_dt,cc_ltv_reference_dt_flag
0,cuid_000001,2015-01-31,cuid_000001,0,2015-01-01,1,1,1000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
1,cuid_000001,2015-02-28,cuid_000001,1,2015-02-01,1,1,5000,0.000000,0,0,platinum,0.000000,classic,classic_platinum,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
2,cuid_000001,2015-03-31,cuid_000001,2,2015-03-01,1,1,2000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
3,cuid_000001,2015-04-30,cuid_000001,3,2015-04-01,1,1,5000,0.000000,0,0,classic,0.000000,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
4,cuid_000001,2015-05-31,cuid_000001,4,2015-05-01,1,1,5000,4301.925916,0,0,classic,86.038518,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,cuid_000100,2021-12-31,cuid_000100,83,2021-12-01,1,1,2000,0.000000,0,0,classic,0.000000,black,black_classic,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,None,0,0,None,0,0,0,2022-04-30,0,0,0.0,None,0
8796,cuid_000100,2022-01-31,cuid_000100,84,2022-01-01,1,1,2000,0.000000,0,0,black,0.000000,classic,classic_black,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,None,0,0,None,0,0,0,2022-04-30,0,0,0.0,None,0
8797,cuid_000100,2022-02-28,cuid_000100,85,2022-02-01,1,1,2000,0.000000,0,0,gold,0.000000,black,black_gold,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# Consolidate Version

In [140]:
def get_globals_paths(repo_path, realm, base_path_name="base_path"):
    globals_paths = {}
    realm_path = repo_path / "realm" / realm
    with open(realm_path / "conf/globals/dev.yaml") as file:
        yaml_file = yaml.load(file, Loader=yaml.FullLoader)
    base_path = yaml_file[base_path_name]
    var_sffx = "${.%s}" % base_path_name
    for k in yaml_file:
        globals_paths[k] = yaml_file[k].replace(var_sffx, base_path)
    
    return globals_paths

def get_data_paths_from_catalog(globals_paths, catalog_path):
    """
        catalog_path: path.to.catalog.yaml
    """
    data_paths = {}
    catalog_dir_path = catalog_path.parent
    with open(catalog_path) as file:
        yaml_file = yaml.load(file, Loader=yaml.FullLoader)
        
    for k0, its0 in yaml_file.items():
        for k1, its1 in its0.items():
            if k1 == "filepath":
                its1_splits = its1.split("/")
                sffx = its1_splits[0]
                for g in globals_paths:
                    if g in sffx:
                        data_paths[k0.split("@")[0]] = "{}/{}".format(globals_paths[g],"/".join(its1_splits[1:]))
    
    return data_paths

def extensive_describe(table):
    print("Display")
    display(table.head())
    for t in ("object", None):
        print("Type: {}".format("numeric" if t is None else t))
        desc = table.describe(include=t)
        desc.loc['dtype'] = table.dtypes
        desc.loc['size'] = len(table)
        desc.loc['nan %'] = table.isnull().mean()
        display(desc)
    return table

def get_table_from_catalog(catalog_path, repo_path=None, table_name=None, realm=None, summary=True):
    realms = ["rb", "cr"]
    table = pd.DataFrame()
    
    if realm is None:
        for r in realms:
            if r in catalog_path.parts:
                realm=r
    realm_path = repo_path / "realm/{}".format(realm)
    globals_paths = get_globals_paths(repo_path=repo_path, realm=realm)
    data_paths = get_data_paths_from_catalog(globals_paths=globals_paths, catalog_path=catalog_path)
    data_path = realm_path / data_paths[table_name]
    table = pd.read_parquet(data_path)
    
    if summary:
        extensive_describe(table)
        
    
    return table

# test
catalog_path = REPO_PATH / "realm/rb/conf/base/fs/catalog.yaml"
table_name = "credit_card"
table = get_table_from_catalog(catalog_path=catalog_path, repo_path=REPO_PATH, table_name=table_name)
table

Display


,_id,_observ_end_dt,customer_id,_observ,_observ_start_dt,cc_customer_exists_flag,cc_credit_card_cnt,cc_sum_credit_card_limit_val,cc_sum_rotation_credit_used_val,cc_nb_stolen_cards_val,cc_nb_blocked_cards_val,cc_product_name_typ,cc_utilization_credit_limit_val,prev_cc_product_name_typ,cc_product_change_typ,cc_black_cnt,cc_classic_cnt,cc_gold_cnt,cc_platinum_cnt,has_black_flag,has_classic_flag,has_gold_flag,has_platinum_flag,cc_black_black_flag,cc_black_classic_flag,cc_black_gold_flag,cc_black_platinum_flag,cc_classic_black_flag,cc_classic_classic_flag,cc_classic_gold_flag,cc_classic_platinum_flag,cc_gold_black_flag,cc_gold_classic_flag,cc_gold_gold_flag,cc_gold_platinum_flag,cc_platinum_black_flag,cc_platinum_classic_flag,cc_platinum_gold_flag,cc_platinum_platinum_flag,cc_avg_transaction_amount_val,cc_sum_transaction_amount_val,cc_transaction_cnt,bpg_services_mcc_cc_avg_transaction_amount_val,bpg_services_mcc_cc_sum_transaction_amount_val,bpg_services_mcc_cc_transaction_cnt,commercial_services_mcc_cc_avg_transaction_amount_val,commercial_services_mcc_cc_sum_transaction_amount_val,commercial_services_mcc_cc_transaction_cnt,stores_mcc_cc_avg_transaction_amount_val,stores_mcc_cc_sum_transaction_amount_val,stores_mcc_cc_transaction_cnt,travel_mcc_cc_avg_transaction_amount_val,travel_mcc_cc_sum_transaction_amount_val,travel_mcc_cc_transaction_cnt,cc_first_reference_dt,has_first_cc_flag,cc_first_reference_dt_flag,cc_second_reference_dt,has_second_cc_flag,cc_second_reference_dt_flag,cc_hard_churn_flag,cc_hard_churn_reference_dt,has_hard_churn_cc_flag,cc_hard_churn_reference_dt_flag,cc_sum_transaction_fee_val,cc_ltv_reference_dt,cc_ltv_reference_dt_flag
0,cuid_000001,2015-01-31,cuid_000001,0,2015-01-01,1,1,1000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
1,cuid_000001,2015-02-28,cuid_000001,1,2015-02-01,1,1,5000,0.000000,0,0,platinum,0.000000,classic,classic_platinum,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
2,cuid_000001,2015-03-31,cuid_000001,2,2015-03-01,1,1,2000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
3,cuid_000001,2015-04-30,cuid_000001,3,2015-04-01,1,1,5000,0.000000,0,0,classic,0.000000,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
4,cuid_000001,2015-05-31,cuid_000001,4,2015-05-01,1,1,5000,4301.925916,0,0,classic,86.038518,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0


Type: object


,_id,_observ_end_dt,customer_id,_observ_start_dt,cc_product_name_typ,prev_cc_product_name_typ,cc_product_change_typ,cc_first_reference_dt,cc_second_reference_dt,cc_hard_churn_reference_dt,cc_ltv_reference_dt
count,8800,8800,8800,8800,4015,4015,4015,7206,7160,3478,5534
unique,100,88,100,88,4,4,16,16,6,9,16
top,cuid_000001,2015-01-31,cuid_000001,2015-01-01,platinum,platinum,platinum_platinum,2022-04-30,2022-04-30,2022-04-30,2022-04-30
freq,88,100,88,100,1698,1743,858,7128,7154,3470,5456
dtype,object,object,object,object,object,object,object,object,object,object,object
size,8800,8800,8800,8800,8800,8800,8800,8800,8800,8800,8800
nan %,0.0,0.0,0.0,0.0,0.54375,0.54375,0.54375,0.181136,0.186364,0.604773,0.371136


Type: numeric


,_observ,cc_customer_exists_flag,cc_credit_card_cnt,cc_sum_credit_card_limit_val,cc_sum_rotation_credit_used_val,cc_nb_stolen_cards_val,cc_nb_blocked_cards_val,cc_utilization_credit_limit_val,cc_black_cnt,cc_classic_cnt,cc_gold_cnt,cc_platinum_cnt,has_black_flag,has_classic_flag,has_gold_flag,has_platinum_flag,cc_black_black_flag,cc_black_classic_flag,cc_black_gold_flag,cc_black_platinum_flag,cc_classic_black_flag,cc_classic_classic_flag,cc_classic_gold_flag,cc_classic_platinum_flag,cc_gold_black_flag,cc_gold_classic_flag,cc_gold_gold_flag,cc_gold_platinum_flag,cc_platinum_black_flag,cc_platinum_classic_flag,cc_platinum_gold_flag,cc_platinum_platinum_flag,cc_avg_transaction_amount_val,cc_sum_transaction_amount_val,cc_transaction_cnt,bpg_services_mcc_cc_avg_transaction_amount_val,bpg_services_mcc_cc_sum_transaction_amount_val,bpg_services_mcc_cc_transaction_cnt,commercial_services_mcc_cc_avg_transaction_amount_val,commercial_services_mcc_cc_sum_transaction_amount_val,commercial_services_mcc_cc_transaction_cnt,stores_mcc_cc_avg_transaction_amount_val,stores_mcc_cc_sum_transaction_amount_val,stores_mcc_cc_transaction_cnt,travel_mcc_cc_avg_transaction_amount_val,travel_mcc_cc_sum_transaction_amount_val,travel_mcc_cc_transaction_cnt,has_first_cc_flag,cc_first_reference_dt_flag,has_second_cc_flag,cc_second_reference_dt_flag,cc_hard_churn_flag,has_hard_churn_cc_flag,cc_hard_churn_reference_dt_flag,cc_sum_transaction_fee_val,cc_ltv_reference_dt_flag
count,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0,8800.0
mean,43.5,0.45625,0.950568,2558.068182,809.70532,0.016705,0.150909,19.060848,0.234318,0.242614,0.232045,0.241591,0.185341,0.192273,0.186477,0.192955,0.011364,0.012841,0.014205,0.017273,0.013636,0.018295,0.022386,0.031477,0.015227,0.02375,0.031023,0.046705,0.016932,0.032386,0.05125,0.0975,8.573295,142.642126,0.147273,6.001553,32.295477,0.032727,5.860368,56.771575,0.060341,5.335446,22.065826,0.021591,5.296644,31.509248,0.032614,0.008864,0.011364,0.000682,0.009773,0.000909,0.000909,0.000909,0.141419,0.009205
std,25.403215,0.498111,1.354174,4015.742507,1855.222661,0.129931,0.416938,55.097911,0.54597,0.552199,0.537085,0.548622,0.388596,0.394109,0.389513,0.39464,0.105999,0.112594,0.11834,0.130293,0.115983,0.134025,0.147945,0.174614,0.122463,0.152278,0.173389,0.211017,0.129023,0.177034,0.22052,0.296654,105.044219,2045.883072,2.076203,87.079386,494.277046,0.480862,82.633998,850.170759,0.871892,81.081891,368.987119,0.33269,79.317192,515.40398,0.487559,0.093734,0.105999,0.026104,0.098378,0.030139,0.030139,0.030139,1.997087,0.095503
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-483.559497,-483.559497,0.0,-449.65219,-449.65219,0.0,-483.559497,-483.559497,0.0,0.0,0.0,0.0,-203.175383,-203.175383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,21.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,43.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,65.25,1.0,2.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,87.0,1.0,8.0,27000.0,19482.264368,2.0,4.0,499.910109,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0

,_id,_observ_end_dt,customer_id,_observ,_observ_start_dt,cc_customer_exists_flag,cc_credit_card_cnt,cc_sum_credit_card_limit_val,cc_sum_rotation_credit_used_val,cc_nb_stolen_cards_val,cc_nb_blocked_cards_val,cc_product_name_typ,cc_utilization_credit_limit_val,prev_cc_product_name_typ,cc_product_change_typ,cc_black_cnt,cc_classic_cnt,cc_gold_cnt,cc_platinum_cnt,has_black_flag,has_classic_flag,has_gold_flag,has_platinum_flag,cc_black_black_flag,cc_black_classic_flag,cc_black_gold_flag,cc_black_platinum_flag,cc_classic_black_flag,cc_classic_classic_flag,cc_classic_gold_flag,cc_classic_platinum_flag,cc_gold_black_flag,cc_gold_classic_flag,cc_gold_gold_flag,cc_gold_platinum_flag,cc_platinum_black_flag,cc_platinum_classic_flag,cc_platinum_gold_flag,cc_platinum_platinum_flag,cc_avg_transaction_amount_val,cc_sum_transaction_amount_val,cc_transaction_cnt,bpg_services_mcc_cc_avg_transaction_amount_val,bpg_services_mcc_cc_sum_transaction_amount_val,bpg_services_mcc_cc_transaction_cnt,commercial_services_mcc_cc_avg_transaction_amount_val,commercial_services_mcc_cc_sum_transaction_amount_val,commercial_services_mcc_cc_transaction_cnt,stores_mcc_cc_avg_transaction_amount_val,stores_mcc_cc_sum_transaction_amount_val,stores_mcc_cc_transaction_cnt,travel_mcc_cc_avg_transaction_amount_val,travel_mcc_cc_sum_transaction_amount_val,travel_mcc_cc_transaction_cnt,cc_first_reference_dt,has_first_cc_flag,cc_first_reference_dt_flag,cc_second_reference_dt,has_second_cc_flag,cc_second_reference_dt_flag,cc_hard_churn_flag,cc_hard_churn_reference_dt,has_hard_churn_cc_flag,cc_hard_churn_reference_dt_flag,cc_sum_transaction_fee_val,cc_ltv_reference_dt,cc_ltv_reference_dt_flag
0,cuid_000001,2015-01-31,cuid_000001,0,2015-01-01,1,1,1000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
1,cuid_000001,2015-02-28,cuid_000001,1,2015-02-01,1,1,5000,0.000000,0,0,platinum,0.000000,classic,classic_platinum,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
2,cuid_000001,2015-03-31,cuid_000001,2,2015-03-01,1,1,2000,0.000000,0,0,classic,0.000000,platinum,platinum_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
3,cuid_000001,2015-04-30,cuid_000001,3,2015-04-01,1,1,5000,0.000000,0,0,classic,0.000000,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
4,cuid_000001,2015-05-31,cuid_000001,4,2015-05-01,1,1,5000,4301.925916,0,0,classic,86.038518,classic,classic_classic,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,2022-04-30,0,0,2022-04-30,0,0,0,2022-04-30,0,0,0.0,2022-04-30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8795,cuid_000100,2021-12-31,cuid_000100,83,2021-12-01,1,1,2000,0.000000,0,0,classic,0.000000,black,black_classic,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,None,0,0,None,0,0,0,2022-04-30,0,0,0.0,None,0
8796,cuid_000100,2022-01-31,cuid_000100,84,2022-01-01,1,1,2000,0.000000,0,0,black,0.000000,classic,classic_black,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,None,0,0,None,0,0,0,2022-04-30,0,0,0.0,None,0
8797,cuid_000100,2022-02-28,cuid_000100,85,2022-02-01,1,1,2000,0.000000,0,0,gold,0.000000,black,black_gold,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0